## 필수과제1
* 실제 데이터셋을 통해 동시 구매 matrix 를 만들고 조건부 확률 기반으로 특정 제품을 지정했을 때 조건부확률 기준 상위 내림차순이 출력될 수 있게 코드로 구현하기

In [1]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import numpy as np

In [2]:
df = pd.read_csv('../dataset/2019-Dec.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3533286 entries, 0 to 3533285
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 242.6+ MB


In [3]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.270,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.050,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.190,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.790,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.560,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


In [4]:
purchase_events = df.loc[df['event_type']=='purchase', ['user_id','product_id']].drop_duplicates()
purchase_events.head()

,user_id,product_id
557,473613801,4554
558,473613801,4600
559,473613801,4607
560,473613801,4640
561,473613801,4689


In [8]:
# 동시 구매이기 때문에 1-user & 1-item 구매 건은 제외
user_item_cnt = purchase_events.groupby('user_id')['product_id'].count()
idx_ = list(user_item_cnt[user_item_cnt==1].index)
multi_purchases = purchase_events.loc[~purchase_events['user_id'].isin(idx_)]
multi_purchases.shape

(204656, 2)

In [9]:
multi_purchases.head()

,user_id,product_id
557,473613801,4554
558,473613801,4600
559,473613801,4607
560,473613801,4640
561,473613801,4689


In [10]:
user_products = multi_purchases.groupby(['user_id'])['product_id'].apply(list)
user_products.head()

user_id
12055855                 [5769907, 5803082, 5651702, 5885811]
15400971                          [5692888, 5694176, 5889691]
25392526                          [5700037, 5605922, 5565812]
28129653    [5565820, 5565822, 5697541, 5725886, 5804464, ...
31647175                                   [5895422, 5810673]
Name: product_id, dtype: object

In [11]:
# itertools 를 활용한 조합 선정과 counting 
from itertools import combinations
from collections import Counter

all_combinations = []
for l in user_products:
    all_combinations +=list(combinations(sorted(l), 2))

pair_counts = dict(Counter(all_combinations))

In [12]:
len(pair_counts)

1767878

In [14]:
pair_counts

{(5651702, 5769907): 1,
 (5651702, 5803082): 1,
 (5651702, 5885811): 1,
 (5769907, 5803082): 3,
 (5769907, 5885811): 1,
 (5803082, 5885811): 1,
 (5692888, 5694176): 1,
 (5692888, 5889691): 1,
 (5694176, 5889691): 1,
 (5565812, 5605922): 1,
 (5565812, 5700037): 6,
 (5605922, 5700037): 1,
 (5565820, 5565822): 4,
 (5565820, 5697541): 1,
 (5565820, 5725886): 1,
 (5565820, 5804464): 1,
 (5565820, 5809249): 1,
 (5565820, 5815163): 1,
 (5565820, 5824841): 1,
 (5565820, 5835147): 1,
 (5565820, 5835158): 1,
 (5565820, 5848909): 1,
 (5565820, 5848910): 1,
 (5565820, 5849998): 1,
 (5565820, 5866172): 1,
 (5565820, 5877599): 1,
 (5565820, 5877603): 1,
 (5565820, 5884021): 1,
 (5565820, 5899825): 1,
 (5565820, 5899840): 1,
 (5565820, 5899841): 1,
 (5565820, 5899842): 2,
 (5565820, 5899845): 1,
 (5565820, 5899926): 1,
 (5565822, 5697541): 1,
 (5565822, 5725886): 2,
 (5565822, 5804464): 1,
 (5565822, 5809249): 1,
 (5565822, 5815163): 1,
 (5565822, 5824841): 1,
 (5565822, 5835147): 1,
 (5565822, 58351

In [13]:
product_counts = multi_purchases['product_id'].value_counts()
product_counts.head()

product_id
5809910    1568
5854897     763
5802432     695
5700037     603
5809912     603
Name: count, dtype: int64

In [15]:
data = []

for (prod1, prod2), count in pair_counts.items():
    #P(prod2|prod1)
    if product_counts[prod1]>0:
        data.append([prod1, prod2, count / product_counts[prod1]])
    if product_counts[prod2]>0:
        data.append([prod2, prod1, count / product_counts[prod2]])

cond_prob_df =pd.DataFrame(data, columns = ['item_A','recommend_item','Confidence'])

In [16]:
cond_prob_df.head()

,item_A,recommend_item,Confidence
0,5651702,5769907,0.021
1,5769907,5651702,0.125
2,5651702,5803082,0.021
3,5803082,5651702,0.015
4,5651702,5885811,0.021


In [17]:
def conditional_prob_recommendation(item_id, k = 5):
    print('>>>>>> Purchase Item Id : {}'.format(item_id))
    rec_item_list = cond_prob_df.loc[cond_prob_df['item_A']==item_id ,:].sort_values(by='Confidence', ascending=False).head(k)
    print('>>>>>> Recommend Item Ids : {}'.format(rec_item_list['recommend_item'].values))
    return rec_item_list

In [18]:
conditional_prob_recommendation(5769907, 5)

>>>>>> Purchase Item Id : 5769907
>>>>>> Recommend Item Ids : [5803082 5808332 5770035 5770034 5770032]


,item_A,recommend_item,Confidence
6,5769907,5803082,0.375
160210,5769907,5808332,0.250
160204,5769907,5770035,0.250
160202,5769907,5770034,0.250
160200,5769907,5770032,0.250


In [19]:
conditional_prob_recommendation(5809912, 5)

>>>>>> Purchase Item Id : 5809912
>>>>>> Recommend Item Ids : [5809910 5809911 5816170 5816172 5816178]


,item_A,recommend_item,Confidence
8131,5809912,5809910,0.524
21351,5809912,5809911,0.247
30898,5809912,5816170,0.149
160778,5809912,5816172,0.071
63238,5809912,5816178,0.068
